In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import plotly.express as px
from io import StringIO

# Step 1: Scrape the Wikipedia Table
def scrape_wikipedia_table(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the first table on the page
    table = soup.find('table')

    if table is None:
        raise ValueError("No tables found on the page")

    # Convert the table to a DataFrame using StringIO
    table_str = str(table)
    df = pd.read_html(StringIO(table_str))[0]

    # Inspect the columns to understand the structure
    print("Column Names in the Table:", df.columns)

    return df

# Step 2: Process the Data (Adjust the columns based on inspection)
def process_data(df):
    # Use the actual column names from the DataFrame
    relevant_columns = ['Location', 'Population']

    # Filter the DataFrame to include only the relevant columns
    df_filtered = df[relevant_columns]
    return df_filtered

# Step 3: Save the Data in a Structured Database (SQLite)
def save_to_database(df, database_name, table_name):
    conn = sqlite3.connect(database_name)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()

# Step 4: Visualize the Data using Plotly
def visualize_data(database_name, table_name, x_column, y_column):
    conn = sqlite3.connect(database_name)
    df = pd.read_sql(f'SELECT * FROM {table_name}', conn)
    fig = px.bar(df, x=x_column, y=y_column, title=f'{y_column} by {x_column}')
    fig.show()
    conn.close()


# Example Usage
if __name__ == "__main__":
    # Define the URL (Example: List of countries by population)
    url = 'https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population'

    # Step 1: Scrape the Wikipedia table
    df = scrape_wikipedia_table(url)

    # Step 2: Process the data
    df_filtered = process_data(df)

    # Step 3: Save the data to an SQLite database
    database_name = 'population_data.db'
    table_name = 'population'
    save_to_database(df_filtered, database_name, table_name)

    # Step 4: Visualize the data using Plotly
    x_column = 'Location'
    y_column = 'Population'
    visualize_data(database_name, table_name, x_column, y_column)

Column Names in the Table: Index(['Unnamed: 0', 'Location', 'Population', '% of world', 'Date',
       'Source (official or from the United Nations)', 'Notes'],
      dtype='object')
